KoGPT2 파인튜닝 https://github.com/SKT-AI/KoGPT2

### 라이브러리 설치

In [1]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 20.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 23.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 45.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 43.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 13.7 MB/s eta 0:00:00


### 라이브러리 불러오기

In [2]:
import pandas as pd
# for padding
# from tensorflow.keras.preprocessing.sequence import pad_sequences

import torch
from torch.utils.data import Dataset, DataLoader
from transformers import AdamW
from tqdm import tqdm

import time
import datetime

### 모델 테스트


In [3]:
from transformers import PreTrainedTokenizerFast
tokenizer = PreTrainedTokenizerFast.from_pretrained("skt/kogpt2-base-v2",
bos_token='</s>', eos_token='</s>', unk_token='<unk>',
pad_token='<pad>', mask_token='<mask>')

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


In [4]:
tokenizer.tokenize("안녕하세요. 한국어 GPT-2 입니다.😤:)l^o")

['▁안녕',
 '하',
 '세',
 '요.',
 '▁한국어',
 '▁G',
 'P',
 'T',
 '-2',
 '▁입',
 '니다.',
 '😤',
 ':)',
 'l^o']

In [5]:
import torch
from transformers import GPT2LMHeadModel

model = GPT2LMHeadModel.from_pretrained('skt/kogpt2-base-v2')

In [6]:
text = '근육이 커지기 위해서는'
input_ids = tokenizer.encode(text, return_tensors='pt')
gen_ids = model.generate(input_ids,
                        max_length=128,
                        repetition_penalty=2.0,
                        pad_token_id=tokenizer.pad_token_id,
                        eos_token_id=tokenizer.eos_token_id,
                        bos_token_id=tokenizer.bos_token_id,
                        use_cache=True)
generated = tokenizer.decode(gen_ids[0])
print(generated)

근육이 커지기 위해서는 무엇보다 규칙적인 생활습관이 중요하다.
특히, 아침식사는 단백질과 비타민이 풍부한 과일과 채소를 많이 섭취하는 것이 좋다.
또한 하루 30분 이상 충분한 수면을 취하는 것도 도움이 된다.
아침 식사를 거르지 않고 규칙적으로 운동을 하면 혈액순환에 도움을 줄 뿐만 아니라 신진대사를 촉진해 체내 노폐물을 배출하고 혈압을 낮춰준다.
운동은 하루에 10분 정도만 하는 게 좋으며 운동 후에는 반드시 스트레칭을 통해 근육량을 늘리고 유연성을 높여야 한다.
운동 후 바로 잠자리에 드는 것은 피해야 하며 특히 아침에 일어나면 몸이 피곤해지기 때문에 무리하게 움직이면 오히려 역효과가 날 수도 있다.
운동을


### 사용자 정의 함수

In [3]:

import re

# 가사 전처리
def preprocess(sentences):
    lst = []
    for sentence in sentences:
        # \n과 \t를 제거한다
        sentence = re.sub('\t', '', sentence)
        sentence = re.sub('\n', '', sentence)
        # 한국어, 공백 빼고 제거하기
        # sentence = re.sub('[^ㄱ-ㅎ가-힣\s]+', '', sentence)
        # 문장 양옆의 띄어쓰기를 지운다
        sentence = sentence.strip()

        lst.append(sentence)
    return lst

### 데이터 불러오기

In [4]:
song_df = pd.read_csv('가사_스테디셀러.csv')
song_df['Lyrics'] = preprocess(song_df['Lyrics'])

In [5]:
song_df['Lyrics']

0       보내야 할 때 보낼 수 있을까쉽게 받아들일 수 있니나는 그게 잘 안돼이별이 그렇게 ...
1       다시 널 사랑할 수 없다면내 모든 아픔까지 가져가네게 익숙해져 버린 나였기에너를 사...
2       해질무렵 날 끌고간 발걸음눈떠보니 잊은줄 알았던 곳에아직도 너의 대한 미움이 남아 ...
3       한참동안 그래 울기만 하잖아오늘도 그리워 하잖아그동안 힘들었지날보며 위로하는 그말 ...
4       늦은밤 들어와 힘에 겨워 지쳐도 보고싶다 달려와 안겨준 사람아파도 내가 걱정돼 한숨...
                              ...                        
2628    잠에서 막 일어난 얼굴도 귀여워잠겨버린 목소리 그것도 귀여워너는 아마 그것까진 모를...
2629    아무말도 없이 날 바라보는게뭔가 이상해서 두귀를 막았어내 눈에 보이는 너의 입모양이...
2630    This is the tihs that y'all ain't ready for Th...
2631    하루 종일 비가 와서니 얼굴이 또 생각 나 눈물이 났어한 걸음 더 걸어가면 잊혀질까...
2632    관둬 가차없이 너랑 끝낼래너같은 남잔 필요없어대체 그럴꺼면 왜 나를 만나니딴 여자랑...
Name: Lyrics, Length: 2633, dtype: object

### DataLoader

In [6]:
lst = ['사랑은 눈에 보이지가 않아어디에 있는지 몰라더듬거리다 찾아다니다이제야 손 끝에 닿았나 봐그때 그대 뒤를 돌아서내게 짓는 웃음에 난 눈을 떠 바람에 흩날리는 그대 향기가낮게 들려오는 그대 목소리가겨울처럼 차가운 내 맘그 안에 스며 들어 와사랑이 눈꽃으로 피어 나그렁 그렁 눈물이 고여이 사랑이 언젠가 시들까 봐바람에 흩날리는 그대 향기가낮게 들려오는 그대 목소리가겨울처럼 차가운 내 맘그 안에 스며 들어 와사랑으로 피어 나하루 이틀 사흘 나흘내 시간을 다 쓸 만큼그대 하나로 난 행복 해바람에 흩날리는 그대 향기가낮게 들려오는 그대 목소리가겨울처럼 차가운 내 맘그 안에 스며 들어 와사랑이 눈꽃으로 피어 나']

In [7]:
len(lst[0]) # 338

338

In [8]:
song_df['Lyrics']

0       보내야 할 때 보낼 수 있을까쉽게 받아들일 수 있니나는 그게 잘 안돼이별이 그렇게 ...
1       다시 널 사랑할 수 없다면내 모든 아픔까지 가져가네게 익숙해져 버린 나였기에너를 사...
2       해질무렵 날 끌고간 발걸음눈떠보니 잊은줄 알았던 곳에아직도 너의 대한 미움이 남아 ...
3       한참동안 그래 울기만 하잖아오늘도 그리워 하잖아그동안 힘들었지날보며 위로하는 그말 ...
4       늦은밤 들어와 힘에 겨워 지쳐도 보고싶다 달려와 안겨준 사람아파도 내가 걱정돼 한숨...
                              ...                        
2628    잠에서 막 일어난 얼굴도 귀여워잠겨버린 목소리 그것도 귀여워너는 아마 그것까진 모를...
2629    아무말도 없이 날 바라보는게뭔가 이상해서 두귀를 막았어내 눈에 보이는 너의 입모양이...
2630    This is the tihs that y'all ain't ready for Th...
2631    하루 종일 비가 와서니 얼굴이 또 생각 나 눈물이 났어한 걸음 더 걸어가면 잊혀질까...
2632    관둬 가차없이 너랑 끝낼래너같은 남잔 필요없어대체 그럴꺼면 왜 나를 만나니딴 여자랑...
Name: Lyrics, Length: 2633, dtype: object

In [9]:
class SongDataset(Dataset):
    def __init__(self, tokenizer, data):
        lst = [str(i) for i in data.values]

        self.item = tokenizer(
            lst,
            return_tensors="pt",
            padding="max_length",
            truncation=True,
            max_length=200,
        )["input_ids"]
        self.length = len(lst)

    def __len__(self):
        return self.length

    def __getitem__(self, idx):
        return self.item[idx]

In [10]:
from transformers import PreTrainedTokenizerFast
tokenizer = PreTrainedTokenizerFast.from_pretrained("skt/kogpt2-base-v2",
                bos_token='</s>', eos_token='</s>', unk_token='<unk>',
                pad_token='<pad>', mask_token='<mask>')

print('Lyrics의 최대 길이: ', max([len(lyrics) for lyrics in song_df['Lyrics']]))

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


Lyrics의 최대 길이:  2501


In [11]:
tokenizer.tokenize('가나다')

['▁가', '나다']

In [12]:
batch_size=16

train_data = SongDataset(tokenizer, song_df['Lyrics'])
train_dataloader = DataLoader(train_data, batch_size=batch_size)

In [13]:
# 데이터셋 잘 만들어졌는지 확인
train_data.__getitem__(3)

tensor([44023, 11097, 10723, 10489, 23001,  9078,  8162,  7965, 30899,  7235,
        27094,  9078,  8162,  7965,  6947, 11097, 20164,  8017,  8263,  7060,
        42282, 12422,  9033,  9022,  7492,  9022,  7492,  8704,  8340, 11097,
        10099,  8143, 31656,  8137, 10437,  7501,  7182,  7141,  8137, 10027,
        18381, 10753, 36101,  7489,  9797,  7788,  8748,  7888, 12582,  8148,
         6855,  8263, 15076, 10099,  8066, 14531,  7652,  7210, 33136,  7760,
         6889, 11608,  6855, 11039,  8263,   415, 29393, 21770, 18896, 16740,
        19490, 15949, 13008, 37090, 11849,   443, 32400,  9456,  6969,  8084,
          426, 14308, 11849,   443, 44389, 10272, 19490, 15949, 13008, 37090,
        11849,   443, 47317, 12964,  9144, 10846,  7335,  9117,  9329,  9117,
         9329,  9117, 50220,  9965, 17380,  7847,  9126,  7185,  9539,  7970,
        14166, 10099,  7470,  9108,  9117,  7652, 12790, 14981,  7055, 12751,
         6853,  7991, 12086, 14042,  6889, 22227,  8263,  8422, 

In [14]:
# 데이터로드 잘 만들어졌는지 확인
dataiter = iter(train_dataloader)
item = next(dataiter)

In [15]:
item

tensor([[11219,  7991,  9337,  ...,  8191, 17748, 40140],
        [ 9427,  9965, 10270,  ..., 34342,  7071,  9548],
        [ 9098,  8267,  7556,  ...,  8705,  9025, 12594],
        ...,
        [30478, 12748, 11382,  ..., 18223,  6872,  8263],
        [24802, 17946, 10247,  ...,  7055,  7235, 13055],
        [10366, 23854,  9769,  ..., 27094,  8265,  9016]])

### GPU 사용

In [16]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print('No GPU available, using the CPU instead.')

There are 1 GPU(s) available.
We will use the GPU: Tesla T4


### 모델 만들기

In [17]:
import torch
from transformers import GPT2LMHeadModel

model = GPT2LMHeadModel.from_pretrained('skt/kogpt2-base-v2').to(device)

In [18]:
model

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(51200, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=51200, bias=False)
)

In [19]:
optimizer = AdamW(model.parameters(),
                  lr = 2e-5,
                  eps = 1e-8)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [20]:
def format_time(elapsed):
    elapsed_rounded = int(round((elapsed)))
    return str(datetime.timedelta(seconds=elapsed_rounded))  # hh:mm:ss

In [21]:
# cuda 메모리 지우기
import torch
torch.cuda.empty_cache()

import gc
gc.collect()

137

In [33]:
def generate(keyword):
    input_ids = tokenizer.encode(keyword, return_tensors='pt')
    input_ids = input_ids.to(device)
    gen_ids = model.generate(input_ids,
                        max_length=128,
                        repetition_penalty=2.0,
                        pad_token_id=tokenizer.pad_token_id,
                        eos_token_id=tokenizer.eos_token_id,
                        bos_token_id=tokenizer.bos_token_id,
                        use_cache=True)
    generated = tokenizer.decode(gen_ids[0])
    return generated

In [47]:
# 학습 전 generate 함수 동작 테스트
keywords = ['밥을 먹어도', '매일같이', '배고파']

for keyword in keywords:
    #keyword = keyword.to(device)
    gen = generate(keyword)
    gen_print = gen.split('\n')[0:2]
    print(f"{keyword} : {gen_print}")

밥을 먹어도 : ['밥을 먹어도 맛있고,', '이렇게 맛있는 #치킨 을 먹고 싶을 때 꼭 먹어봐야겠어요!']
매일같이 : ['매일같이 씩 웃으며 "우리도 잘해줘서 고맙다"고 말했다.', '이날 행사에는 박근혜 대통령, 김무성 새누리당 대표와 원유철 원내대표, 정병국 문화체육관광방송통신위원장, 이한구 정책위의장 등 여야 지도부와 각 당 사무총장이 참석했다.']
배고파 : ['배고파서 #먹스타그램', '<unk>beats_diet</d> 오늘은 내일도 좋은 하루 되세요!']


In [48]:
# 학습 모드
model.train()
epochs = 15
min_loss = 1

for epoch in range(epochs):
    print('======== Epoch {:} / {:} ========'.format(epoch + 1, epochs))
    t0 = time.time()
    total_loss = 0

    for input_text in tqdm(train_dataloader):
        input_tensor = input_text.to(device)
        # print(input_tensor)
        # print(input_tensor.shape)

        # input과 label이 같다.
        outputs = model(input_tensor, labels=input_tensor)
        # print('outputs:', outputs)
        loss = outputs[0]

        # cpu로 변경하기
        input_tensor.cpu()

        optimizer.zero_grad()
        model.zero_grad()
        loss.backward()
        optimizer.step()


    print(f"epoch {epoch} loss {outputs[0].item():0.2f}")
    print("  Training epcoh took: {:}".format(format_time(time.time() - t0)))


    keywords = ['밥을 먹어도', '매일같이', '배고파']
    for keyword in keywords:
        gen = generate(keyword)
        gen_print = gen.split('\n')[0:2]
        print(f"{keyword} : {gen_print}")


======== Epoch 1 / 15 ========


100%|██████████| 165/165 [02:13<00:00,  1.24it/s]


epoch 0 loss 3.25
  Training epcoh took: 0:02:13
밥을 먹어도 : ['밥을 먹어도 돼요 괜히 웃어봐도 너무 예뻐서 눈물이 날까 봐 울지 말아줘야 해요. 내 맘이 이렇게 아파 보여 사랑해 난 너를 좋아하지만 그게 싫어서 못살아 보려고 해도 안돼 니가 왜 이러니 내가 어떻게 살아야 하는 건데 이럴 수가 없어 그냥 널 보내줄래 나만 있으면 되잖니가 정말 미안해서 그래그래너무 보고 싶었나보니까 네 모습이 너무나 아름다워보여 눈이 부시네눈물까지 흘렸는데그냥 볼 수 있어내 안에 있는 모든 게 다 들어있을 거라고 생각했는데내가 얼마나']
매일같이 : ['매일같이 웃고 있는 너의 모습그냥 보고 싶었어너무나도 사랑했던 그 사람아직까지도 내겐 너무 잘 어울리는데요이렇게 행복하게 살고 있어내게 다가와줘서 고마워하지만 난 아직 미련만 남아있어사랑해 넌 정말 좋은 사람이야난 아직도 못 잊혀져버렸지오늘은 또 다른 날 만나러 가야 돼지금부터라도 다시 시작해야돼우리 모두 함께 할 수 있게다시 한번 더 말해봐그래 그래 오늘만은 꼭꼭 돌아와주길 바래다시는 안 올 거라고생각했었는데왜 이렇게 힘들기만 해니가 없는 건']
배고파 : ['배고파도 못 견디는다던 그 사람그런데 왜 난 너를 사랑해요사랑은 뭔지 몰라서 나는 너무 슬퍼너무 아파내 맘을 아프게 해줘이제 그만 날 떠나가세요.내가 아닌 다른 사람이 돼버렸어왜 나만 이렇게 아픈거니난 니와 같은 사랑을 원했나봐니가 없는건 아니야아픈 내 가슴엔 사랑이 없어눈물처럼 눈물이 흘러넘쳐오네널 보고 싶었다면미련 없이 널 볼 수 있다면정말 미안한 마음뿐이야우리 함께라면 행복할텐데,그래야만 내가 더 잘']
======== Epoch 2 / 15 ========


100%|██████████| 165/165 [02:18<00:00,  1.19it/s]


epoch 1 loss 2.93
  Training epcoh took: 0:02:19
밥을 먹어도 : ['밥을 먹어도 괜찮아요그냥 그냥 그게 좋아요.이렇게 말해도 돼서 난 너를 사랑해줘도 될 것 같아그래야 해줄 수 있어너무나 잘해주지 못해 미안한 마음뿐이야내가 너무 보고 싶어서 그래 내가 널 원해서 그랬어니가 날 버린 건 아니니까널 위해서라도 니가 내겐 필요했잖니내 맘을 돌려주겠다네사랑한다 말해봐 봐라말하지 말아달란 말이 없어난 네 눈빛에 빠져버렸는데네가 나를 버렸다고 해도넌 내게 필요한 거 아냐고눈물만 흘리고 있죠']
매일같이 : ['매일같이 널 보면은근히 웃음이 나와서웃는 얼굴도내겐 너무 예쁜데그냥 내 맘에 들어요이렇게 너를 사랑해줘너무 예뻐 보여줄래난 네가 좋아보여널 볼 때면눈물이 나네그래 그래 난 니만 봐봐넌 정말 잘생겼는데왜 날 이렇게 빤하게 쳐다봤어니가 먼저 고백했잖아니야 내가 잘못됐나보이지 않아내가 왜 이러는지알고 있단 말이라도 해볼까오늘따라 더 울적해져날 보고 싶어서사랑한다고 말해줬더니네가 또 나를 원해서더 많이 아파']
배고파 : ['배고파서 울어요그대 눈물이 그댈 아프게 했죠아픈 맘에 눈물만 나네요.사랑한다 말했었나 봐도눈물보다 더 아픈 건가봐이렇게라도 사랑해줘야 할텐데왜 이렇게 못난 내맘을 몰라주던 너의 모습이 너무 그리워울지 말아줄래너를 떠나면 안돼내 곁에 있어주면 돼괜찮다고 말해줬잖니내가 널 보내준다면널 위해 기도할께니가 날 버린건 잘못인걸알면서 왜 그렇게 애써 외면하는지넌 정말 미안한 걸 알면서도 난 잘 모르겠다']
======== Epoch 3 / 15 ========


...

epoch 12 loss 1.49
  Training epcoh took: 0:02:20
밥을 먹어도 : ['밥을 먹어도 배가 고파서 먹지 않는 라면 수저를 입에 물고 줬다 하면 밥은 잘 먹는지 물도 안 먹고 비만 맞춰 준다고 해서 친구는 대접해 주고 가끔씩엔 커피를 마셔주며 환하게 웃어주고 때로는 미안한 마음에도 위로가 되어 주지만 그게 행복인 거야 하지만 난 너의 행복을 위해서라면 모든 것을 버려야 해 이대로 영원히 함께 살아야 돼 서로 믿고 의지하며 살아왔던 날들이 너무 많아서 그런 만남이 아닌가 싶기도 하고 이런 날이 올 줄 알았었는데 이렇게라도 꼭 다시 만나야만 하는 이유와 또 다른 이유로 인해 어쩔 수가 없는']
매일같이 : ['매일같이 계속 니 생각만하고 있어 넌 나를 떠나버릴것 같아 내 맘을 숨긴 채 난 너의 뒤에서 날 기다렸어 네게로 와줘 나의 곁에 머물러줄래 나 혼자만의 비밀 하나를 알려 줄 수 없어 이 비밀을 알고 싶어 이렇게 널 기다리고 있잖아 그 누구도 나에게 말하지 못할 단 하나뿐인걸 알기에 하지만 지금 내게 필요한 건 오직 그것밖에 없는데 사랑이란 그런거야 잠시라도 멈출수가 없는 거라서 조금은 망설였지만 그래도 언젠간 너도 함께 할수 있다면 이런 기분 좋은 날이 올꺼라고 믿고있어 시간이 지나면 나도 조금씩 잊혀져 갈 꺼라는']
배고파 : ['배고파도 울지말아요그대곁에 잠들기전에내게 돌아와줘서 고마워요한번만 더 나를 봐달라고세상 모든걸 다 줄수있게너와 함께면 된다고난 말했었죠이렇게 힘든날 모두잊을수가 있을까내가 할 수 있는건줄 알았었는데왜 이렇게 힘들기만 했는지모르겠어 난 너무 슬퍼져서멀리 떠나가버린 너를그리움에 지쳐버렸네슬픔은 그대에게로온몸을 던져준것 그대가 너무나 보고싶어서혼자 남겨둔채떠나가지 말아달라는 말을끝없이 하고픈']
======== Epoch 14 / 15 ========


100%|██████████| 165/165 [02:20<00:00,  1.18it/s]


epoch 13 loss 1.36
  Training epcoh took: 0:02:20
밥을 먹어도 : ['밥을 먹어도 돼요  그대가 없으면 난 정말 싫어요. 그대만 생각하면 미칠 것 같죠 내 맘을 숨기고 싶지 않아. 이대로 영원히 함께 할 수 없다면 나를 떠나 다른 사랑 찾아도 좋아, 너를 위해서라면 다 해줄게 나 아닌 사람인데 왜 내게 이러는지 모르겠어 나는 울고 있어 어떻게든 행복하게 살거야 내가 가진 모든걸 가져가 어떡하면 좋겠냐 묻자 웃으면서 얘기해 줄테니 걱정하지마 지금껏 잘 살아왔잖아 미안하단 말도 하지 말아줘 그저 편히 눈 감는 날 용서할께 널 위해 살아가도록 꼭']
매일같이 : ['매일같이 맑은 아침에 잠 못 들고 깨고 싶은 생각만 잔뜩 떠오르면 난 거울에 비친 내 모습 그대로인 걸 왜 이리도 어색한지 너를 생각하면 할수록 자꾸 눈물이 나는걸 널 보면 하는 말이라서 이렇게 한 번 해볼래 니가 보고 싶단 말은 하지마 I wanna be your love  나 오늘밤은 너무 힘들 것 같아 미칠 것만 같애 지금 이 순간부터 네게 보여줄 수 있는 건 오직 그것뿐이야 내가 아닌 다른 사람이라도 내게 이런 말을 하면 조금 더 위로라도 될까 걱정돼 그냥 솔직히 말해봐 그런 날이면']
배고파 : ['배고파서 술을 마셨어요즘은 술이 좀 취했나봐그냥 술 한잔 걸치고 괜히 외롭죠오늘도 그대와밤을 보내며 난 하루종일 고민만 했었지난 오늘 밤새워둔 전화에 울리는 목소리사양해 줄 수 있는 건 다 해줄께 있잖아한번쯤 너의 전화번호 다시 물어도 대답해줄게전화할 때 왜 그래왜 안돼니 맘이 급해서 그랬는데너 때문에 또 후회하고 싶어서그래 그렇게 살았었는데정말 미안하긴 하더라 나 혼자 너무 힘들었던 거야술 없이는']
======== Epoch 15 / 15 ========


100%|██████████| 165/165 [02:20<00:00,  1.18it/s]


epoch 14 loss 1.28
  Training epcoh took: 0:02:20
밥을 먹어도 : ['밥을 먹어도 눈치 채지 못해요 그대만 보면 난 미칠 것 같아그대에게 잘해주지도 못할뿐이죠 그댄 날 아프게 하지 말아줘 내겐 아픈사랑을 가르쳐 준 사람입니다 hey 나를 사랑해봐도 될까 봐서 모르겠어요. 나 미워지면 안돼울고 싶으면 울면 돼 이대로 갈 수 밖에 없잖아. 제발 부탁이야 내가 아파하면 못난 애가 되주는 사람이 되어주세요가끔은 혼자라도 편히 웃자던 그런 말보다 더 듣고 싶은 말이 있어 다른 사람과 비교하지마 내게 필요한건 오직 하나야 널']
매일같이 : ['매일같이 울기만 하는 날엔더 할 일이 없어그냥 멍하니 너의 생각만나는 그대로 있어이대로 널 보낼 수 있다면너를 다시 만날 순 없지만난 아직까진 그런게 아닌데도널 걱정하는 내 맘은늘 그래왔어 더 이상 흔들릴 것 같아하지만 넌 그럴 필요없잖아언제라도 내게 말해주겠니너의 마음까지가끔 니 생각에 잠들곤 해내가 미안해 하지마나의 마음을 알아봐알고 싶지만 너무 궁금해서말하지 않아도 돼니가 나를 좋아한다해도왜 그땐 몰랐을런 말을 해야하는지 모르겠니아님 오시면 꼭']
배고파 : ['배고파서 밥 먹었어밥도 잘 못먹었는데술 없이는 괜시리 지겹게 생각했나봐그냥 그런거야아니면 내가 바보였는지너를 사랑하게 된 것도 아닌데왜 너만 미워하니내가 너무 미웠지이렇게 널 많이 좋아한 이유는 네가 있어서일까 나는 니 맘에 들어맞는 거라고 해말 안해도 알 수 없는 내 마음이라며 날 위로해줘요오래 전 우리처럼다시 만날 순 없지만난 아직 그걸로 할께요.참 고마워.니가 없이 난 살아있어널 위해 살수밖에 없었단']


In [51]:
# 학습 후 generate 함수 동작 테스트
keywords = ['밥을 먹어도', '매일같이', '배고파']

for keyword in keywords:
    #keyword = keyword.to(device)
    gen = generate(keyword)
    gen_print = gen.split('\n')[0:2]
    print(f"{keyword} : {gen_print}")

밥을 먹어도 : ['밥을 먹어도 배가 아플 땐도시락 챙겨 먹어요저녁에 먹는 밥도둑이 잦다거나 화를 낼 때면밥만 잘먹어주면 돼아무것도 안 먹고 자다가 깨어나보면뭐가 그리 바쁜지그냥 그냥 지나쳐줘오늘은 이대로 내버려둬우리 함께 거닐던 거리엔새벽 세시 반잠을 자는 너를둔 채밤 늦게까지 잠들게 해왜냐고 왜 나를 잊으려 하지도 않잖아이쿠 오늘따라 더 신나봐잘못된 생각 때문에혼자서 집을 나서는건좀 무리한 일인걸I']
매일같이 : ['매일같이 넌 항상 웃기만 해아무리 그댈 달래도난 듣지 않고매일 같이 울어도 난 믿지 않아이젠 말할 수 있어요그대 눈 앞에서 숨소리만 들릴 뿐이지혹시 내 맘을 아프게 할까봐잠깐이나 한 번 말해 봐서말해줄래너가 날 좋아한다고내가 널 사랑할 줄은 몰랐어하지만 너는 알고 있잖아이런 게 설레지면 안돼걱정하지마사랑해 니 옆에 있는걸널 볼 수만 있다면내 마음껏 행복하게 해주렴오늘밤 한번 더 내게 달려와줘가슴속에 숨겨둔 말들을 꺼내날 믿어준']
배고파 : ['배고파 너를 안고서 잤다고 깨워달라고 말하지 않았잖아 잠들때까지 혼자 있었냐며 내게 말했어 내가 잘못 했니 니가 너무 보고 싶어서 한 얘기야 내일이면 다 알것 같아서 오늘도 전화 안받는데 왜 또 이러는지 정말 답답해 널 많이 좋아한다 해서 그게 행복인줄 몰랐었지 그래서 미안하단 말도 못했다 말을 해봐라 라면 그래 그렇게 쉽게 말해줘 너도 날 사랑하면 할수록 난 더 그리워져 가슴이 아파 울다가 다시 눈물이 나와 바보 같은 나 이젠 말할래 우리 이렇게 만나니까 서로 잘']


In [53]:
keywords = ['나 혼자', '학원에서', '감정과는']

for keyword in keywords:
    #keyword = keyword.to(device)
    gen = generate(keyword)
    gen_print = gen.split('\n')[0:2]
    print(f"{keyword} : {gen_print}")

나 혼자 : ['나 혼자 울고 있네요  그대가 너무 그리워서 하루도 못 자겠죠 내일이면 다시 볼 수 있을까봐 오늘은 말할께 잠시라도 그대에게 난 기대 잠들 수도 있게 이대로 날 사랑 할수만 있다면 그때는 말야 내가 먼저 너를 떠나간대도 니가 없는 곳 정말로 널 잊을수가 없어서 이렇게 눈물로 삼켜본다면 너무나 많이 아파해야 했나 봐 미칠것 같아서 정말 미치도록 보고 싶었었는데 왜 말하지 못했니 지금 말하는건데 아니면 내게 말해줘 꼭 한번 만나볼래 시간이 되면 차마 전하지 못할 나의 슬픈 얘기 나를 떠난다는']
학원에서 : ['학원에서 치킨을 먹었죠 그댈 만나고 싶다고 했네요그대에게 난 정말 싫은걸까 그랬더니 어휴 바보같다 하더라구여자는 왜 안모르는 척내겐 절대이렇게 모독만 하는지어떻게 알았냐는 핀잔에웃으며 인사했었지만 왠일인지 나 혼자 왔는데왜 멀쩡한데 웃기만 하나있느냐며헤어진후에야우리 헤어지고 싶은거 맞나봐아니면 너 아니면못살것 같은데이런 만남으로 또 다시날 사랑하게 될 줄이야너무 보고싶어서 그래']
감정과는 : ['감정과는 달랐어요 나만 사랑했었죠 그댈 놓지 못한 내 마음 때문에 난 울고 있었네요. 이대로 영원히 함께 할래면 아직은 그대 곁에 있으니까. 잠시 스쳐간 바람처럼 괜찮아 보여도 될 거라고 생각했는데 왜 이렇게 멀리서 바라볼 수 있나 봐 두렵기도 했었는데 오오 너와 내가 너무 달라서 더 잘할 수가 없는데 자꾸 눈물이 나와 그냥 하는 말이라 생각하고 있지만 언젠가 다시 보게 되겠지만 그래도 알고 있어 그렇게 쉽게 끝날 순 없어 미안해 하지마 가지 말아줘 제발 이런 슬픈 말은 안해도 돼 이별이']


In [49]:
epoch

14

In [50]:
# 모델 저장
torch.save(model.state_dict(), f"kogpt2_fine-tunning_{epoch}.pt")

In [ ]:
# 모델 로드
# model.load_state_dict(torch.load("kogpt2_fine-tunning_14.pt"))